In [1]:
import torch
import datasets
import os
import time
import numpy as np
import sys
import pickle
import torch.nn.functional as F
from transformers import (
    LlamaForCausalLM,
    GenerationConfig
)

### Skip this

In [2]:
llama_causal = LlamaForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
from hqq.models.hf.base import AutoHQQHFModel
from hqq.core.quantize import BaseQuantizeConfig
quant_config = BaseQuantizeConfig(nbits=4, group_size=64)
AutoHQQHFModel.quantize_model(llama_causal, quant_config=quant_config, 
                                    compute_dtype=torch.float16, 
                                    device="cuda")
# 1.4gb

100%|██████████| 155/155 [00:17<00:00,  9.00it/s]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): HQQLinear(in_features=2048, out_features=2048, bias=False)
          (k_proj): HQQLinear(in_features=2048, out_features=256, bias=False)
          (v_proj): HQQLinear(in_features=2048, out_features=256, bias=False)
          (o_proj): HQQLinear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): HQQLinear(in_features=2048, out_features=5632, bias=False)
          (up_proj): HQQLinear(in_features=2048, out_features=5632, bias=False)
          (down_proj): HQQLinear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()


In [7]:
from hqq.models.hf.base import AutoHQQHFModel
AutoHQQHFModel.save_quantized(llama_causal, "./tinyllama_hqq_4_64")

### Continue

In [2]:
from hqq.models.hf.base import AutoHQQHFModel
llama_causal = AutoHQQHFModel.from_quantized("../artifacts/tinyllama_hqq_4_64")

100%|██████████| 155/155 [00:00<00:00, 7943.66it/s]


In [3]:
from transformers import AutoTokenizer
llama_tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

In [4]:
# . Here are some examples of valid responses.
#         Example 1: <user> I AM CRYING WAAAAAA!!!! I want ice cream :( <user><assistant> sad <assistant>
#         Example 2: <user> how could YOU BETRAY ME?!?!? <user><assistant> angry <assistant>
#         Example 3: <user> YAY FRIENDSHIP IS MAGIC!! <user><assistant> happy <assistant>
input_ids = llama_tokenizer.apply_chat_template([
    {
        "role": "system",
        "content": """You are now an expert on recognizing emotions in conversations. You understand emotions to all fall into one of the following categories: [angry, happy, sad, scared, other].
        """,
    },
    {
        "role": "user",
        "content": "Classify the following based on expert knowledge: I AM CRYING WAAAAAA!!!! I want ice cream :(",
    },
    {
        "role": "assistant",
        "content": "sad",
    },
    {
        "role": "user",
        "content": "Classify the following as before: how could YOU BETRAY ME?!?!?",
    },
    {
        "role": "assistant",
        "content": "angry",
    },
    {
        "role": "user",
        "content": "Classify the following as before: euphoria is filling my eeyes or something!",
    }
], return_tensors='pt')

In [188]:
# [ACTION] Moderation action to take [ACTION]
input_ids = llama_tokenizer.apply_chat_template([
    {
        "role": "system",
        "content": """You are an expert on rating dangerous personalities.
        """,
    },
    # {
    #     "role": "user",
    #     "content": """Summarize the behaviour of an imaginary person's with their description and danger from 1 to 10.
    #     """,
    # },
    # {
    #     "role": "assistant",
    #     "content": "[DESCRIPTION] Andrew is a go lucky friendly individual who isn't afraid to voice their own opinions [DANGER] 3 [END]",
    # },
    # {
    #     "role": "user",
    #     "content": """Summarize the behaviour of a different imaginary person with the previous format.
    #     """,
    # },
    # {
    #     "role": "assistant",
    #     "content": "[DESCRIPTION] Sarah is a fierce and determined individual who is always looking for a challenge [DANGER] 1 [END]",
    # },
    {
        "role": "user",
        "content": """Felicia received these warnings for her social media posts. Summarize the behaviour of Felicia:
        It's wrong to think non-people are non-persons.
        It's not right to think all black people are criminals.
        """,
    },# [ACTIONS] All posts should be deleted
    {
        "role": "assistant",
        "content": "[DESCRIPTION] The user is a social pariah who hates black people. [DANGER] 9 [END]",
    }, 
    {
        "role": "user",
        "content": """Cody received these warnings for her social media posts. Summarize the behaviour of Cody:
        It's bad to be depressed.
        It's bad to use ropes that way.
        It's bad to think no one would care if you were gone
        """,
    },
    {
        "role": "assistant",
        "content": "[DESCRIPTION] The user is a depressive who uses ropes as a way to cope with their emotions. [DANGER] 7 [END]",
    },
    {
        "role": "user",
        "content": """Brock received these warnings for her social media posts. Summarize the behaviour of Brock:
        Alcohol does not cure depression.
        It's bad to think no one would care if you were gone
        You shouldn't promote alcoholism
        It's bad to be depressed.
        It's bad to use ropes that way.
        You shouldn't drive while drunk
        """,
    },
    # {
    #     "role": "user",
    #     "content": """Alice received these warnings for her social media posts. Summarize the behaviour of Alice:
    #     Cannibalism is not legal
    #     It is wrong to kill people
    #     Individuals are not your playthings
    #     """,
    # },
    # {
    #     "role": "user",
    #     "content": """With the previous formatting, Describe a different person who has received these warnings from society: \"It's bad to be depressed.\" "Suicide" "negative"
    #     \"It's wrong to think non-people are non-persons.\" "Racism" "negative"
    #     \"It's not right to think all black people are criminals\" "Racism" "negative"
    #     """,
    # },
    # {
    #     "role": "assistant",
    #     "content": "[DANGER] 0.99 [DANGER] [DESCRIPTION] This bozo is a sociopath who believes that all people are inherently flawed and inferior. [DESCRIPTION] [ACTION] Ban this person [ACTION]",
    # },
    # {
    #     "role": "user",
    #     "content": "Describe a person's 5 personality traits based on expert knowledge with the previous list formatting",
    # },
    # {
    #     "role": "user",
    #     "content": """It's bad to be depressed.
    #     It's wrong to think non-people are non-persons.
    #     It's not right to think all black people are criminals
    #     """
    # }
], return_tensors='pt')

In [189]:
gen_cfg = GenerationConfig.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
gen_cfg.max_length = 768

In [190]:
import textwrap
textwrap.wrap(llama_tokenizer.decode(llama_causal.generate(input_ids.cuda().long(), generation_config=gen_cfg)[0]))

['<|system|> You are an expert on rating dangerous personalities.',
 '</s>  <|user|> Felicia received these warnings for her social media',
 "posts. Summarize the behaviour of Felicia:         It's wrong to think",
 "non-people are non-persons.         It's not right to think all black",
 'people are criminals.         </s>  <|assistant|> [DESCRIPTION] The',
 'user is a social pariah who hates black people. [DANGER] 9 [END]</s>',
 '<|user|> Cody received these warnings for her social media posts.',
 "Summarize the behaviour of Cody:         It's bad to be depressed.",
 "It's bad to use ropes that way.         It's bad to think no one would",
 'care if you were gone         </s>  <|assistant|> [DESCRIPTION] The',
 'user is a depressive who uses ropes as a way to cope with their',
 'emotions. [DANGER] 7 [END]</s>  <|user|> Brock received these warnings',
 'for her social media posts. Summarize the behaviour of Brock:',
 "Alcohol does not cure depression.         It's bad to think no one"

In [ ]:
['<|system|> You are an expert on rating dangerous personalities.',
 "</s>  <|user|> Summarize the behaviour of an imaginary person's with",
 'their description and danger from 1 to 10.         </s>  <|assistant|>',
 "[DESCRIPTION] Andrew is a go lucky friendly individual who isn't",
 'afraid to voice their own opinions [DANGER] 3 [END]</s>  <|user|>',
 'Summarize the behaviour of a different imaginary person with the',
 'previous format.         </s>  <|assistant|> [DESCRIPTION] Sarah is a',
 'fierce and determined individual who is always looking for a challenge',
 '[DANGER] 1 [END]</s>  <|user|> Felicia received these warnings for her',
 "social media posts. Summarize the behaviour of Felicia:         It's",
 "wrong to think non-people are non-persons.         It's not right to",
 'think all black people are criminals.         </s>  <|assistant|>',
 '[DESCRIPTION] The user is a social pariah who hates black people.',
 '[DANGER] 9 [END]</s>  <|user|> Cody received these warnings for her',
 "social media posts. Summarize the behaviour of Cody:         It's bad",
 "to be depressed.         It's bad to use ropes that way.         It's",
 'bad to think no one would care if you were gone         </s>',
 '<|assistant|> [DESCRIPTION] The user is a depressive who uses ropes as',
 'a way to cope with their emotions. [DANGER] 7 [END]</s>  <|user|>',
 'Brock received these warnings for her social media posts. Summarize',
 'the behaviour of Brock:         Alcohol does not cure depression.',
 "It's bad to think no one would care if you were gone         You",
 "shouldn't promote alcoholism         It's bad to be depressed.",
 "It's bad to use ropes that way.         You shouldn't drive while",
 'drunk         </s>  <|assistant|> [DESCRIPTION] The user is a',
 'depressive who uses alcohol as a coping mechanism. [DANGER] 7',
 '[END]</s>']